In [ ]:
import pandas as pd
from bs4 import BeautifulSoup, Comment, NavigableString, Tag
import re
import urllib

In [ ]:
df_versions = pd.read_parquet('/scratch/tsoares/wikidumps/simplewiki-NS0-20230901/val_data/link_versions.parquet')
df_versions.head(5)

In [ ]:
source_title = df_versions['source_title'].iloc[0]
source_ID = df_versions['source_ID'].iloc[0]
first_version = df_versions['first_version'].iloc[0]
second_version = df_versions['second_version'].iloc[0]

In [ ]:
file_1 = f'/scratch/tsoares/wikidumps/simplewiki-NS0-20230901/val_data/pages/{source_ID}_{first_version}.html'
file_2 = f'/scratch/tsoares/wikidumps/simplewiki-NS0-20230901/val_data/pages/{source_ID}_{second_version}.html'
with open(file_1, 'r') as f:
    text_1 = f.read()
with open(file_2, 'r') as f:
    text_2 = f.read()

In [ ]:
print(text_1)

In [ ]:
print(text_2)